In [116]:
#import libraries
import cvxpy as cp
import numpy as np
import pandas as pd

#load data
xls = pd.ExcelFile('data\case14.xlsx')
branch = pd.read_excel(xls, 'branch')
gen = pd.read_excel(xls, 'gen')
bus = pd.read_excel(xls, 'bus')
cost = pd.read_excel(xls, 'cost')

#Y matrix
base = 100
bus_len = bus.shape[0]
gen_len = len(gen['bus'].ravel())

Ybus = np.matrix(np.zeros([bus_len,bus_len]),dtype=complex)
shunt = np.zeros(bus_len, dtype=complex)
x = [pd.DataFrame() for i in range(bus_len)]
for i in range(bus_len):  
    x[i] = branch.loc[branch['fbus'] == i]
    fro = [j for j in x[i]['tbus']]
    fro = pd.DataFrame(fro)
    for j in range(len(fro)):
        Ybus[i-1,fro.loc[j][0]-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        Ybus[fro.loc[j][0]-1,i-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])

for i in range(bus_len):
    Ybus[i,i] = np.sum(Ybus[i,:])*(-1) + complex(0,sum(branch.loc[(branch['fbus']==i+1) | (branch['tbus']==i+1)]['b'])/2)

W = cp.Variable((2*bus_len, 2*bus_len), PSD=True)

e = ['' for i in range(bus_len)]
ee= ['' for i in range(bus_len)]
for i in range(bus_len):
    e[i] = np.matrix(np.zeros(bus_len))
    e[i][0,i] = 1
    ee[i] = np.transpose(e[i])*e[i]


l = [['' for i in range(len(bus))] for i in range(len(bus))]
L = [['' for i in range(len(bus))] for i in range(len(bus))]
L_ = [['' for i in range(len(bus))] for i in range(len(bus))]
M = ['' for i in range(len(bus))]
N = ['' for i in range(len(bus))]
p = ['' for i in range(len(bus))]
q = ['' for i in range(len(bus))]
for i in range(len(bus)):
    M[i] = np.concatenate([np.concatenate([ee[i],np.matrix(np.zeros([bus_len,bus_len]))],axis=1),
               np.concatenate([np.matrix(np.zeros([bus_len,bus_len])),ee[i]],axis=1)])
    N[i] = np.concatenate([np.concatenate([np.matrix(np.zeros([bus_len,bus_len])),np.matrix(np.zeros([bus_len,bus_len]))],axis=1),
               np.concatenate([np.matrix(np.zeros([bus_len,bus_len])),ee[i]],axis=1)])
    p[i] = 0.5*np.concatenate([np.concatenate([np.real(np.transpose(ee[i]*Ybus) + ee[i]*Ybus),np.imag(np.transpose(ee[i]*Ybus) - ee[i]*Ybus)],axis=1),
                             np.concatenate([np.imag(ee[i]*Ybus - np.transpose(ee[i]*Ybus)), np.real(np.transpose(ee[i]*Ybus) + ee[i]*Ybus)],axis=1)])
    q[i] = -0.5*np.concatenate([np.concatenate([np.imag(np.transpose(ee[i]*Ybus) + ee[i]*Ybus),np.real(ee[i]*Ybus - np.transpose(ee[i]*Ybus))],axis=1),
                             np.concatenate([np.real(np.transpose(ee[i]*Ybus) - ee[i]*Ybus), np.imag(np.transpose(ee[i]*Ybus) + ee[i]*Ybus)],axis=1)])

for i in range(len(branch)):
    f = int(branch.iloc[i]['fbus'])
    t = int(branch.iloc[i]['tbus'])
    r = branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['r'][i]
    x = branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['x'][i]
    l[f-1][t-1] = (complex(0,branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['b'][i])/2 + 1/(complex(r,x)))*np.transpose(e[f-1])*e[f-1] - 1/(complex(r,x))*np.transpose(e[f-1])*e[t-1]
    l[t-1][f-1] = (complex(0,branch.loc[(branch['fbus']==f) & ((branch['tbus']==t))]['b'][i])/2 + 1/(complex(r,x)))*np.transpose(e[t-1])*e[t-1] - 1/(complex(r,x))*np.transpose(e[t-1])*e[f-1]
    L_[f-1][t-1] = -0.5*np.concatenate([np.concatenate([np.imag(l[f-1][t-1] + np.transpose(l[f-1][t-1])),np.real(-np.transpose(l[f-1][t-1]) + l[f-1][t-1])],axis=1),
                             np.concatenate([np.real(-l[f-1][t-1] + np.transpose(l[f-1][t-1])), np.imag(l[f-1][t-1] + np.transpose(l[f-1][t-1]))],axis=1)])
    L_[t-1][f-1] = -0.5*np.concatenate([np.concatenate([np.imag(l[t-1][f-1] + np.transpose(l[t-1][f-1])),np.real(-np.transpose(l[t-1][f-1]) + l[t-1][f-1])],axis=1),
                             np.concatenate([np.real(-l[t-1][f-1] + np.transpose(l[t-1][f-1])), np.imag(l[t-1][f-1] + np.transpose(l[t-1][f-1]))],axis=1)])
    L[f-1][t-1] = 0.5*np.concatenate([np.concatenate([np.real(l[f-1][t-1] + np.transpose(l[f-1][t-1])),np.imag(np.transpose(l[f-1][t-1]) - l[f-1][t-1])],axis=1),
                             np.concatenate([np.imag(l[f-1][t-1] - np.transpose(l[f-1][t-1])), np.real(l[f-1][t-1] + np.transpose(l[f-1][t-1]))],axis=1)])
    L[t-1][f-1] = 0.5*np.concatenate([np.concatenate([np.real(l[t-1][f-1] + np.transpose(l[t-1][f-1])),np.imag(np.transpose(l[t-1][f-1]) - l[t-1][f-1])],axis=1),
                             np.concatenate([np.imag(l[t-1][f-1] - np.transpose(l[t-1][f-1])), np.real(l[t-1][f-1] + np.transpose(l[t-1][f-1]))],axis=1)])
    
# declare variables
Pd = np.zeros(bus_len)
Qd = np.zeros(bus_len)
Vmax = np.zeros(bus_len)
Vmin = np.zeros(bus_len)
Pmax = np.zeros(bus_len)
Pmin = np.zeros(bus_len)
Qmax = np.zeros(bus_len)
Qmin = np.zeros(bus_len)
cost_cons = np.zeros(gen_len)
cost_lin = np.zeros(gen_len)
cost_quad = np.zeros(gen_len)

#load data into variables
for i in range(bus_len):
    Pd[i] = bus['Pd'][i]/base
    Qd[i] = bus['Qd'][i]/base
    Vmax[i] = bus['Vmax'][i]
    Vmin[i] = bus['Vmin'][i]

for i in range(gen_len):
    k = i
    i = gen['bus'].ravel()[i]
    Pmax[i-1] = gen.loc[gen['bus'] == i]['Pmax'].ravel()[0]/base
    Pmin[i-1] = gen.loc[gen['bus'] == i]['Pmin'].ravel()[0]/base
    Qmax[i-1] = gen.loc[gen['bus'] == i]['Qmax'].ravel()[0]/base
    Qmin[i-1] = gen.loc[gen['bus'] == i]['Qmin'].ravel()[0]/base
    
    
for i in range(gen_len):
    cost_cons[i] = cost['cons'][i]
    cost_lin[i] = cost['lin'][i]*base
    cost_quad[i] = cost['quad'][i]*base**2
    

Pg = [0 for i in range(gen_len)]
Qg = [0 for i in range(gen_len)]
for i in range(gen_len):
    Pg[i] = cp.trace(p[gen['bus'][i]-1]@W) + Pd[gen['bus'][i]-1]
    Qg[i] = cp.trace(q[gen['bus'][i]-1]@W) + Qd[gen['bus'][i]-1]


#Constraints
constraints = [(W >> 0)]
constraints += [W[:,bus_len] == 0]
constraints += [W[bus_len,:] == 0]
   
for i in range(bus_len):
    constraints += [cp.trace(M[i]@W) <= Vmax[i]**2]
    constraints += [cp.trace(M[i]@W) >= Vmin[i]**2]
    constraints += [cp.trace(p[i]@W) <= Pmax[i]-Pd[i]]
    constraints += [cp.trace(p[i]@W) >= Pmin[i]-Pd[i]]
    constraints += [cp.trace(q[i]@W) <= Qmax[i]-Qd[i]]
    constraints += [cp.trace(q[i]@W) >= Qmin[i]-Qd[i]]
 
#Flow constraints   
for i in range(len(branch)):
    f = int(branch.iloc[i]['fbus'])
    t = int(branch.iloc[i]['tbus']) 
    val = (branch.loc[i]['rateA']/base)
    if(val == 0):
        val = 100
    constraints += [cp.trace(L[f-1][t-1]@W)**2 + cp.trace(L_[f-1][t-1]@W)**2 <= (val)**2]
    constraints += [cp.trace(L[t-1][f-1]@W)**2 + cp.trace(L_[t-1][f-1]@W)**2 <= (val)**2]
    
#Transforming the cost as an SOCP constraint    
# alpha = ["" for i in range(gen_len)]
# gencost = 0
# for i in range(gen_len):
#     alpha[i] = cp.Variable()
#     A = np.array([cost_lin[i],2*np.sqrt(cost_quad[i])])
#     b = 1+cost_cons[i]-alpha[i]
#     c = -cost_lin[i]
#     d = 1-cost_cons[i]+alpha[i]
#     constraints += [cp.SOC(c*Pg[i]+d,A*Pg[i]+b)]
#     gencost = gencost + alpha[i]
    
# objective = cp.Minimize(gencost)

# Cost function
objective = cp.Minimize(sum(cost_cons) + 
    (sum(Pg[i]*cost_lin[i] for i in range(gen_len))) + 
    (sum(cost_quad[i]*(Pg[i]**2) for i in range(gen_len)))
                       )


#Objective function
prob = cp.Problem(objective, constraints)

# solve
prob.solve(solver=cp.MOSEK)#, verbose=True)


8082.442149999853

In [117]:
[evl,evc] = np.linalg.eigh(W.value)
eigWratio = evl[-1]/evl[-2]
Vopt = np.sqrt(evl[-1])*evc[:,-1]
Vopt = Vopt[0:bus_len] + [complex(0,Vopt[i]) for i in range(bus_len,2*bus_len)]
abs(np.around(Vopt,decimals=4))

array([1.06      , 1.04004401, 1.01654653, 1.01350841, 1.01870134,
       1.013751  , 1.01590157, 1.0558268 , 0.9937124 , 0.98927582,
       0.99782231, 0.99764783, 0.99233765, 0.97392716])

In [118]:
-Vopt

array([-1.06001688-0.j        , -1.03754854+0.0726695j ,
       -1.00125279+0.17541371j, -1.00200825+0.15225991j,
       -1.01012137+0.13206133j, -0.98680948+0.23216598j,
       -0.99595668+0.20009885j, -1.03800814+0.19315791j,
       -0.96679596+0.2296802j , -0.96133943+0.23362323j,
       -0.96985523+0.23441716j, -0.96740197+0.24379969j,
       -0.96209067+0.24313814j, -0.9414353 +0.24957824j])